In [21]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from google.colab import files
import time
import struct
import PIL
import torchvision.models as models
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
gpu_boole = torch.cuda.is_available()
print(gpu_boole)

True


In [23]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip tiny-imagenet-200.zip

import io
import glob
import os
from shutil import move
from os.path import join
from os import listdir, rmdir

target_folder = './tiny-imagenet-200/val/'
test_folder   = './tiny-imagenet-200/test/'

#os.mkdir(test_folder)
val_dict = {}
with open('./tiny-imagenet-200/val/val_annotations.txt', 'r') as f:
    for line in f.readlines():
        split_line = line.split('\t')
        val_dict[split_line[0]] = split_line[1]
        
paths = glob.glob('./tiny-imagenet-200/val/images/*')
for path in paths:
    file = path.split('/')[-1]
    folder = val_dict[file]
    if not os.path.exists(target_folder + str(folder)):
        os.mkdir(target_folder + str(folder))
        os.mkdir(target_folder + str(folder) + '/images')
    if not os.path.exists(test_folder + str(folder)):
        os.mkdir(test_folder + str(folder))
        os.mkdir(test_folder + str(folder) + '/images')
        
        
for path in paths:
    file = path.split('/')[-1]
    folder = val_dict[file]
    if len(glob.glob(target_folder + str(folder) + '/images/*')) <25:
        dest = target_folder + str(folder) + '/images/' + str(file)
    else:
        dest = test_folder + str(folder) + '/images/' + str(file)
    move(path, dest)
    
rmdir('./tiny-imagenet-200/val/images')

Streaming output truncated to the last 5000 lines.
  inflating: tiny-imagenet-200/val/images/val_3979.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3963.JPEG  
  inflating: tiny-imagenet-200/val/images/val_7199.JPEG  
  inflating: tiny-imagenet-200/val/images/val_2752.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9687.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9407.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3603.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3412.JPEG  
  inflating: tiny-imagenet-200/val/images/val_6982.JPEG  
  inflating: tiny-imagenet-200/val/images/val_8496.JPEG  
  inflating: tiny-imagenet-200/val/images/val_7332.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9241.JPEG  
  inflating: tiny-imagenet-200/val/images/val_4196.JPEG  
  inflating: tiny-imagenet-200/val/images/val_5980.JPEG  
  inflating: tiny-imagenet-200/val/images/val_6697.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9969.JPEG  
  inflating: tiny-ima

In [24]:
#ImageNet dataloading taken from 
#https://github.com/tjmoon0104/pytorch-tiny-imagenet/blob/master/AlexNet.ipynb
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
    ])
}

data_dir = 'tiny-imagenet-200/'
num_workers = {
    'train' : 100,
    'val'   : 0,
    'test'  : 0
}
image_datasets = {x: dataset.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val','test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=100,
                                             shuffle=True, num_workers=num_workers[x])
                  for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

In [25]:


for i, data in enumerate(dataloaders['test']):
  images, labels = data
  print(labels)

tensor([  6,   0,   0,  97,  29, 146,   0, 190,   0,   0,   0,  35,   0,  52,
        131,   3,  84, 171, 183,  34,  65,   0,  38,  48,   0, 195,   0,   0,
          0,   0,   0,  57, 180,  74,   0,   0,   0, 167,   0,   0,   0,   0,
          2,   0,  18, 134, 168, 105,  78,  27,   0, 161, 190, 155,  86,   0,
          0, 185,   0, 160, 181,   0,   0,  50, 115, 178,  40,   0,   0,  64,
          0, 127,  34,  65,   0,  36, 107,   0,   0, 196,   0,  11,   0,   6,
         13,   0,  33,   0,   0,  29,  59,   0,   0,   0,   0, 169,  30, 149,
         41,  55])
tensor([  0,  26,   0,   0, 151, 188,  47,  12,   0,   6,  80,   0,   0,   0,
          0,  57,   0, 162,   0,   0,  57,   0,  82,  58,  99,   0, 163, 176,
          0,  91,   0,  51,  65,   0,   0,  25, 114,  56, 138,   0, 104,   0,
        110,   0, 178,   0,  19,   0,   0, 116,   0, 156,   0,   0, 165,  13,
          0,   0,   0,   3, 164,   0,   0,   0,   0, 194,   0, 129, 106,   0,
         54,  24,  38, 109,   0,  74,   0, 15

In [27]:
CUDA_LAUNCH_BLOCKING=1

In [28]:
netparam = models.vgg19_bn(pretrained=True)
netparam.classifier[-1] = nn.Linear(in_features=4096, out_features=200)
#for param in netparam.parameters():
  #param.requires_grad = False
#for layer in netparam.classifier:
  #layer.requires_grad = True
print(netparam.parameters)
optimizer = torch.optim.SGD(netparam.parameters(), lr=0.001, momentum=0.9)
#optimizer = torch.optim.Adam(netparam.parameters(), 0.001)
loss_metric = torch.nn.CrossEntropyLoss()
if gpu_boole:
 netparam = netparam.cuda()

<bound method Module.parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mo

RuntimeError: ignored

In [ ]:
def train_eval(netparam, verbose = 1):
    correct = 0
    total = 0
    loss_sum = 0
    for images, labels in dataloaders['train']:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()
        outputs = netparam(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.float() == labels.float()).sum()
        loss_sum += loss_metric(outputs,labels).detach()
        
    if verbose:
        print('Train accuracy: %f %%' % (100.0 * correct / total))
        print('Train loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return 100.0 * correct / total, loss_sum.cpu().data.numpy().item() / total
    
def test_eval(netparam, verbose = 1):
    correct = 0
    total = 0
    loss_sum = 0
    for images, labels in dataloaders['test']:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()    
        outputs = netparam(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.float() == labels.float()).sum()
        loss_sum += loss_metric(outputs,labels).detach()

    if verbose:
        print('Test accuracy: %f %%' % (100.0 * correct / total))
        print('Test loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return 100.0 * correct / total, loss_sum.cpu().data.numpy().item() / total

def validation_eval(netparam, verbose = 1):
    correct = 0
    total = 0
    loss_sum = 0
    for images, labels in dataloaders['val']:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()
        outputs = netparam(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.float() == labels.float()).sum()
        loss_sum += loss_metric(outputs,labels).detach()
        
    if verbose:
        print('Validation accuracy: %f %%' % (100.0 * correct / total))
        print('Validation loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return 100.0 * correct / total, loss_sum.cpu().data.numpy().item() / total
    '''
    correct = 0
    total = 0
    loss_sum = 0
    for images, labels in dataloaders['val']:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()
        outputs = netparam(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.float() == labels.float()).sum()
        loss_sum += loss_metric(outputs,labels).detach()

    if verbose:
        print('Validation accuracy: %f %%' % (100.0 * correct / total))
        print('Validation loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return 100.0 * correct / total, loss_sum.cpu().data.numpy().item() / total
    '''

In [ ]:
epochs = 10

#defining batch train loss recording arrays for later visualization/plotting:
loss_batch_store = []
train_accuracy_scores = []
test_accuracy_scores = []
validation_accuracy_scores = []
training_loss = []
epoch_list = [i for i in range(1, epochs + 1)]
epoch_loss = []
maxvalacc = 0
print("Starting Training")
#training loop:
for epoch in range(epochs):
  time1 = time.time() #timekeeping
  for i, (x, y) in enumerate(dataloaders['train']):
    if (i % 100 ==0 ):
      print(i)
    if gpu_boole:
      x = x.cuda()
      y = y.cuda()
    if i > 0 or epoch > 0:
      optimizer.zero_grad()
    outputs = netparam.forward(x)
    loss = loss_metric(outputs,y)
    loss.backward()
    if i > 0 or epoch > 0:
      loss_batch_store.append(loss.cpu().data.numpy().item())
    optimizer.step()
    epoch_loss.append(loss.detach())
    

  print("Epoch",epoch+1,':')
  train_perc, train_loss = train_eval(netparam)
  valid_perc, valid_loss = validation_eval(netparam)
  test_perc, test_loss = test_eval(netparam)
  #if valid_perc > maxvalacc:
   # maxvalacc = valid_perc
    #torch.save(netparam.state_dict(), path) 
  validation_accuracy_scores.append(valid_perc)
  time2 = time.time() #timekeeping
  print('Elapsed time for epoch:',time2 - time1,'s')
  print('ETA of completion:',(time2 - time1)*(epochs - epoch - 1)/60,'minutes')
  print()

## Plotting batch-wise train loss curve:
#plt.plot(epoch_list, training_loss, '-o', label = 'train_loss', color = 'blue')
#plt.xlabel('Epoch Number')
#plt.ylabel('Training Loss at Given Epoch')
#plt.legend()
#plt.show()
#plt.plot(epoch_list, train_accuracy_scores, color = 'blue')
#plt.plot(epoch_list, test_accuracy_scores, color = 'blue')
#plt.plot(epoch_list, validation_accuracy_scores, color = 'blue')
#plt.xlabel('Epoch Number')
#plt.ylabel('Accuracy in Percent')
test_perc_a, test_loss_a = test_eval(netparam)